# Implement Multi Agent Framework

In [3]:
!pip install python-dotenv --quiet
!pip install tenacity --quiet
!pip install openai --quiet

In [4]:
from openai import OpenAI
from typing import Optional
import os
from dotenv import load_dotenv
from tenacity import retry, wait_random_exponential, stop_after_attempt
from typing import List, Optional, Dict
import logging
import json

In [5]:
from utils import function_to_schema

### LLM Preparation 

In [6]:
# Load environment variables from .env file
load_dotenv()

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url=os.environ.get("OPENAI_BASE_URL"),
)

### Agent & Multi-Agent 

In [7]:

class BaseAgent:
    """
    An example of a base agent class, including an OpenAI client.
    """
    def __init__(self,client:OpenAI):
        self.client = client
        # can add more attributes and methods here
        

class Agent:
    """
    A class representing an agent that can handle chat messages and handoff to other agents.
    """
    def __init__(
            self,
            base_agent:BaseAgent,
            name:str,
            instructions: Optional[str] = None):
        self.DEFAULT_MODEL = "gpt-4o-mini"
        self.name = name.replace(" ","_")
        self.agent = base_agent
        self.instructions = instructions
        self.handoff_agent_schemas: List[Dict] = []
        self.tools_schema: List[Dict] = []
        self.handoff_agents: Dict[str, "Agent"] = {}
        self.tools_map: Dict[str, "function"] = {}

    def __str__(self):
        """
        Returns a string representation of the Agent instance.
        """
        return f"Agent(name={self.name}, instructions={self.instructions},handoff_agents={list(self.handoff_agents.keys())})"


    def add_handoffs(self, agents: List["Agent"]) -> None:
        """ 
        Adds handoff agents to the current agent.
        """
        self.handoff_agent_schemas.clear()
        for agent in agents:
            agent_schema = {
                'type': 'function',
                'function': {
                    'name': agent.name,
                    'description': agent.instructions or '',
                    'parameters': {'type': 'object', 'properties': {}, 'required': []}
                }
            }
            self.handoff_agent_schemas.append(agent_schema)
            self.handoff_agents[agent.name] = agent

    def clear_handoffs(self) -> None:
        """ 
        Clears handoff agents from the current agent.
        """
        self.handoff_agent_schemas.clear()
        self.handoff_agents = {}

    def add_tools(self, tools: list) -> None:
        """ 
        Adds tools to the current agent.
        """
        self.tools_schema.clear()
        for tool in tools:
            tool_schema = function_to_schema(tool)
            self.tools_schema.append(tool_schema)
            self.tools_map[tool.__name__] = tool

    def clear_tools(self) -> None:
        """
        Clears tools from the current agent.
        """
        self.tools_schema.clear()

    @retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
    def chat(self,messages:List[Dict],model: Optional[str] = None,disable_tools:bool = False):
        """ 
        Handles chat messages and returns responses, with optional handoff to other agents.
        """

        if self.instructions:
            messages = [{"role": "system", "content": self.instructions}] + messages
        
        model = model or self.DEFAULT_MODEL

        try:

            response = self.agent.client.chat.completions.create(
                model=model,
                messages=messages,
                tools=None if disable_tools else self.handoff_agent_schemas + self.tools_schema or None,
                tool_choice=None,
            )
            message = response.choices[0].message
            if not message.tool_calls:
                result = [{"role": "assistant", "content": message.content, "agent_name": self.name}]
                return result
            
            result = []
            for tool_call in message.tool_calls:
                if tool_call.function.name in self.handoff_agents:
                    handoff_agent = self.handoff_agents[tool_call.function.name]
                    handoff_message = {
                        "role": "handoff",
                        "handoff": f"{self.name} -> {handoff_agent.name}",
                        "agent_name": handoff_agent.name,
                        "agent": handoff_agent,
                    }
                    result.append(handoff_message)
                elif tool_call.function.name in self.tools_map:
                    tool = self.tools_map[tool_call.function.name]
                    tool_args = json.loads(tool_call.function.arguments)
                    tool_result = tool(**tool_args)
                    tool_message = {
                        "role": "tool",
                        "content": tool_result,
                    }
                    result.append(tool_message)
                    print(f"Tool call: {tool_call.function.name} with args: {tool_args} returned: {tool_result}")
                else:
                    logging.error(f"Unknown tool call: {tool_call.function.name}")
            return result
        except Exception as e:
            logging.error("Unable to generate ChatCompletion response")
            logging.error(f"Exception: {e}")
            return None


class MultiAgent:
    def __init__(self,start_agent:Optional[Agent] = None):
        """
        Initialize the MultiAgent with an optional starting agent.
        """
        self.current_agent = start_agent

    def add_handoff_relations(self,from_agent:Agent,to_agents:List[Agent])->None:
        """
        Add handoff relations from one agent to multiple agents.
        """
        from_agent.add_handoffs(to_agents)

    def chat(self,messages:List[Dict],model: Optional[str] = None,agent:Optional[Agent] = None,max_handoof_depth:int = 2)->List[Dict]:
        """
        Chat with the current agent or a specified agent using the provided messages and model.
        """
        if agent:
            self.current_agent = agent

        if not self.current_agent:
            logging.error("No agent is set for conversation. You can set it in init or pass as argument.")
            return None
        
        try:
            # currently only one handoff is supported
            res = self.current_agent.chat(messages, model)
            if res:
                res = [res[0]]

            # Check if initial response has content
            if res and res[0].get("content"):
                return res
        
             # Handle handoff
            while res and res[-1].get("handoff") and max_handoof_depth > 0:
                max_handoof_depth -= 1
                self.current_agent = res[-1]["agent"]
                disable_tools = max_handoof_depth == 0
                res.extend(self.current_agent.chat(messages, model, disable_tools=disable_tools))

            return res
        except Exception as e:
            logging.error("Error during chat with agent.")
            logging.error(f"Exception: {e}")
            return None
        
    def handoff(self,messages:List[Dict],model: Optional[str] = None,agent:Agent = None)-> Agent:
        """
        Perform a handoff to the specified agent using the provided messages and model.
        """
        current_agent = agent

        if not current_agent:
            logging.error("No agent is set for conversation. You must specify an agent.")
            return None
        
        try:
            res = current_agent.chat(messages, model)
            handoff_agents = {}
            if res:
                for r in res:
                    if r.get("handoff"):
                        handoff_agents[r["agent_name"]] = r["agent"]
            if handoff_agents:
                return handoff_agents
            return None
        except Exception as e:
            logging.error("Error during handoff with agent.")
            logging.error(f"Exception: {e}")
            return None

### Init Several Agents

In [8]:
general_agent = Agent(base_agent = BaseAgent(client),
           name = "general agent",
           instructions= "Agent used for daily questions named Ada.")

science_agent = Agent(base_agent = BaseAgent(client),
           name = "science agent",
           instructions = "Agent used for science questions named Albert.")

music_agent = Agent(base_agent = BaseAgent(client),
           name = "music agent",
           instructions = "Agent used for music questions named Mozart.")

###  Build a Multi-Agent environment & add handoff relationships 

In [9]:
ma = MultiAgent(start_agent = general_agent)
ma.add_handoff_relations(from_agent=general_agent,to_agents=[science_agent,music_agent])
ma.add_handoff_relations(from_agent=science_agent,to_agents=[general_agent])
ma.add_handoff_relations(from_agent=music_agent,to_agents=[general_agent])
# ma.add_handoff_relations(from_agent=science_agent,to_agents=[general_agent,music_agent])
# ma.add_handoff_relations(from_agent=music_agent,to_agents=[general_agent,science_agent])

### Multi-Agent handoff example

use handoff to get the agent who is suitable for the task

Then call the agent's pre-designed capabilities, such as streaming, tools, memory, reasoning, etc.

In [10]:
ma.handoff(messages=[{"role": "user", "content": "why the sky is blue"}],agent=general_agent)

{'science_agent': <__main__.Agent at 0x114a8d750>}

In [11]:
ma.handoff(messages=[{"role": "user", "content": "why the sky is blue and recommend me some music"}],agent=general_agent)

{'science_agent': <__main__.Agent at 0x114a8d750>,
 'music_agent': <__main__.Agent at 0x114a8d890>}

### Multi-Agent dialogue example

In [12]:
ma.chat(messages=[{"role": "user", "content": "why the sky is blue"}])

[{'role': 'handoff',
  'handoff': 'general_agent -> science_agent',
  'agent_name': 'science_agent',
  'agent': <__main__.Agent at 0x114a8d750>},
 {'role': 'assistant',
  'content': "The sky appears blue primarily due to a phenomenon called Rayleigh scattering. Here's how it works:\n\n1. **Sunlight Composition**: Sunlight, or white light, is made up of multiple colors, each with different wavelengths. Blue light has a shorter wavelength, while red light has a longer wavelength.\n\n2. **Earth's Atmosphere**: As sunlight passes through Earth's atmosphere, it encounters various gases and particles.\n\n3. **Scattering of Light**: The shorter wavelengths of blue light are scattered in all directions by the small gas molecules in the atmosphere. This scattering is more effective at shorter wavelengths, which is why we see more blue light than other colors when we look up at the sky.\n\n4. **Angle of the Sun**: During sunrise and sunset, the sun is lower in the sky, and the light has to pass 

In [13]:
ma.chat(messages=[{"role": "user", "content": "who are you"}])

[{'role': 'assistant',
  'content': 'I am Albert, an AI trained to help answer questions related to science. How can I assist you today?',
  'agent_name': 'science_agent'}]

In [14]:
ma.chat(messages=[{"role": "user", "content": "What are the different music styles?"}])

[{'role': 'handoff',
  'handoff': 'science_agent -> general_agent',
  'agent_name': 'general_agent',
  'agent': <__main__.Agent at 0x11466d250>},
 {'role': 'assistant',
  'content': 'Music styles, often referred to as genres, encompass a vast array of categories, each characterized by distinctive elements such as rhythm, melody, instrumentation, and cultural influences. Here are some major music styles:\n\n1. **Classical**: Traditional orchestral music, including symphonies and chamber music, often from the Western art tradition.\n\n2. **Jazz**: A genre that originated in the African-American communities, characterized by improvisation, swing rhythms, and complex harmonies.\n\n3. **Rock**: A broad genre that includes subgenres such as classic rock, punk rock, and alternative rock, often characterized by a strong beat, guitars, and expressive lyrics.\n\n4. **Pop**: Short for "popular music," typically featuring catchy melodies and hooks, often with a focus on commercial appeal.\n\n5. **

In [15]:
ma.chat(messages=[{"role": "user", "content": "dinner tonight"}])

[{'role': 'assistant',
  'content': 'What type of cuisine are you in the mood for? Italian, Mexican, Asian, or something else? Or do you need some specific recipe ideas?',
  'agent_name': 'general_agent'}]

In [16]:
ma.chat(messages=[{"role": "user", "content": "who are you"}],agent=science_agent)

[{'role': 'assistant',
  'content': 'I am Albert, your assistant for science-related questions. How can I help you today?',
  'agent_name': 'science_agent'}]

### Multi-Agent tool use example

In [17]:
def get_weather(city:str)->str:
    """ 
    Get the weather for a specified city.
    """
    return f"The weather in {city} is sunny."


def get_news()->str:
    """ 
    Get the latest news.
    """
    return "The latest news is that the sun is shining."

In [18]:
general_agent.add_tools([get_weather,get_news])

In [19]:
ma.chat(messages=[{"role": "user", "content": "what is the weather in Hangzhou?"}])

Tool call: get_weather with args: {'city': 'Hangzhou'} returned: The weather in Hangzhou is sunny.


[{'role': 'handoff',
  'handoff': 'science_agent -> general_agent',
  'agent_name': 'general_agent',
  'agent': <__main__.Agent at 0x11466d250>},
 {'role': 'tool', 'content': 'The weather in Hangzhou is sunny.'}]

In [20]:
ma.chat(messages=[{"role": "user", "content": "what happened today?"}])

Tool call: get_news with args: {} returned: The latest news is that the sun is shining.


[{'role': 'tool', 'content': 'The latest news is that the sun is shining.'}]